In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [2]:

def selectkbest(X, y, k):
    selector = SelectKBest(score_func=chi2, k=k)
    X_new = selector.fit_transform(X, y)
    return X_new

def split_scale(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=0
    )
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test

def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return acc, cm, report

def logistic(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    return evaluate(model, X_test, y_test)
def svm_linear(X_train, y_train, X_test, y_test):
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)
    return evaluate(model, X_test, y_test)

def svm_rbf(X_train, y_train, X_test, y_test):
    model = SVC(kernel='rbf')
    model.fit(X_train, y_train)
    return evaluate(model, X_test, y_test)

def knn(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X_train, y_train)
    return evaluate(model, X_test, y_test)

def naive(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    return evaluate(model, X_test, y_test)

def decision(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier(criterion='entropy')
    model.fit(X_train, y_train)
    return evaluate(model, X_test, y_test)

def random_forest(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train, y_train)
    return evaluate(model, X_test, y_test)


In [4]:
dataset = pd.read_csv("prep.csv")
dataset = pd.get_dummies(dataset, drop_first=True)


In [5]:
X = dataset.drop("classification_yes", axis=1)
y = dataset["classification_yes"]


In [6]:
X_kbest = selectkbest(X, y, 5)

X_train, X_test, y_train, y_test = split_scale(X_kbest, y)


In [7]:
results = {}

results["Logistic"] = logistic(X_train, y_train, X_test, y_test)[0]
results["SVM Linear"] = svm_linear(X_train, y_train, X_test, y_test)[0]
results["SVM RBF"] = svm_rbf(X_train, y_train, X_test, y_test)[0]
results["KNN"] = knn(X_train, y_train, X_test, y_test)[0]
results["Naive Bayes"] = naive(X_train, y_train, X_test, y_test)[0]
results["Decision Tree"] = decision(X_train, y_train, X_test, y_test)[0]
results["Random Forest"] = random_forest(X_train, y_train, X_test, y_test)[0]


result = pd.DataFrame(
    [[
        results["Logistic"],
        results["SVM Linear"],
        results["SVM RBF"],
        results["KNN"],
        results["Naive Bayes"],
        results["Decision Tree"],
        results["Random Forest"]
    ]],
    index=["ChiSquare"],
    columns=["Logistic", "SVML", "SVMNL", "KNN", "Naive", "Decision", "Random"]
)



In [8]:
result

,Logistic,SVML,SVMNL,KNN,Naive,Decision,Random
ChiSquare,0.94,0.94,0.95,0.89,0.83,0.95,0.95
